In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

In [5]:
!pip install  ntlk 


ERROR: Could not find a version that satisfies the requirement ntlk (from versions: none)
ERROR: No matching distribution found for ntlk


In [ ]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset, load_metric
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import ntlk
from ntlk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
import pandas as pd

ntlk.download("punkt")

In [ ]:
model_cpkt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_cpkt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_cpkt).to(device)

In [ ]:
!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip summarizer-data.zip

In [ ]:
dataset_samsum = load_from_disk('samsum_dataset')

In [ ]:
split_lengths = [len(dataset_samsum[split] ) for split in dataset_samsum]
print(f"Split length: {split_lengths} " )
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDiaglogue")

print(dataset_samsum["test"][1]["dialogue"])
print("\nSummary")
print(dataset_samsum["test"][1]["summary"])

In [ ]:
def convert_to_features(example_batch):
    input_encoding = tokenizer(example_batch['dialogue'], max_length=512, truncation=True)
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length=512, truncation=True)
    return {
        'input_ids': input_encoding['input_ids'],
        'attention_mask': input_encoding['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [ ]:
dataset_samsum_pt = dataset_samsum.map(convert_to_features, batched=True)

In [ ]:
#Training
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model= model_pegasus)

In [ ]:
from transformers import TrainingArguments, Trainer
trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_gpu_eval_batch_size=1,
    weight_decay=0.01, logging_steps = 10, 
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [ ]:
trainer = Trainer(model= model_pegasus, args=trainer_args, 
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"], 
                  eval_dataset=dataset_samsum_pt["validation"])

In [ ]:
trainer.train()

In [ ]:
#Evalution
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """Split the dataset into smaller batches that we can process 
    Yield Successive batch-sized chunks from list_of_elements """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i:i+batch_size]


def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, batch_size =16, device = device,
                                column_text="article", column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size ))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer(article_batch, max_length = 512, truncation=True, padding ="max_length", return_tensors="pt")

        summaries  = model.generate(input_ids= inputs["input_ids"].to(device),
                                    attention_mask = inputs["attention_mask"]. to(device),
                                    lenght_penalty=0.8, num_beans=8, max_length=128)
        
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokeniztion_spaces=True) for s in summaries]
        decoded_summaries = [d.replace("" , "") for d in decoded_summaries]
        metric.add_batch(prediction=decoded_summaries, references = target_batch)
    
    #Finally compute and return the ROUGE Scores
    
    score = metric.compute()
    return score

In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

In [ ]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size =2, 
    column_text='dialogue', column_summary='summary'
)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=[f'pegausus'])

In [ ]:
##Save the model
model_pegasus.svae_pretrained("pegasus-samsum-model")
##Save tokenizer
tokenizer.save_pretrained("tokenzier")

In [ ]:
#load
tokenizer = AutoTokenizer.from_pretrained("/content/tokenzier")

In [ ]:
#Prediction
gen_kwargs = {"lenght_penalty":0.8, "num_beams":8, "max_length": 128}

sample_text = dataset_samsum["test"][0]["dialogue"]
reference = dataset_samsum["test"][0]["summary"]
pipe = pipeline("summarization", model ="pegasus-samsum-model", tokenizer=tokenizer)

##
print("Dialogue")
print(sample_text)

print("\nReferences")
print(reference)
print("\nModel Summary")
print(pipe(sample_text,**gen_kwargs)[0]["summary_text"])
